In [1]:
import glob, os, json, random, base64, sys, yaml, math
import numpy as np
import shutil
import faulthandler

import queue
from tqdm import tqdm
try:
    sys.path.append(glob.glob('../carla/dist/carla-*%d.%d-%s.egg' %(
    sys.version_info.major,
    sys.version_info.minor,
    'win-amd64' if os.name =='nt' else 'linux-x86_64')))
except IndexError:
    pass


# -- imports -------------------------------------------------------------------

In [2]:
import carla
from carla import ColorConverter as cc
import cv2

import random

try:
    import pygame
    from pygame.locals import K_ESCAPE
    from pygame.locals import K_SPACE
    from pygame.locals import K_a
    from pygame.locals import K_d
    from pygame.locals import K_s
    from pygame.locals import K_w
except ImportError:
    raise RuntimeError('cannot import pygame, make sure pygame package is installed')

try:
    import numpy as np
except ImportError:
    raise RuntimeError('cannot import numpy, make sure numpy package is installed')


class CarlaSyncMode(object):
    """
    Context manager to synchronize output from different sensors. Synchronous
    mode is enabled as long as we are inside this context

        with CarlaSyncMode(world, sensors) as sync_mode:
            while True:
                data = sync_mode.tick(timeout=1.0)

    """

    def __init__(self, world, *sensors, **kwargs):
        self.world = world
        self.sensors = sensors
        self.frame = None
        self.delta_seconds = 1.0 / kwargs.get('fps', 20)
        self._queues = []
        self._settings = None

    def __enter__(self):
        self._settings = self.world.get_settings()
        self.frame = self.world.apply_settings(carla.WorldSettings(
            no_rendering_mode=False,
            synchronous_mode=True,
            fixed_delta_seconds=self.delta_seconds))

        def make_queue(register_event):
            q = queue.Queue()
            register_event(q.put)
            self._queues.append(q)

        make_queue(self.world.on_tick)
        for sensor in self.sensors:
            make_queue(sensor.listen)
        return self

    def tick(self, timeout):
        self.frame = self.world.tick()
        data = [self._retrieve_data(q, timeout) for q in self._queues]
        assert all(x.frame == self.frame for x in data)
        return data

    def __exit__(self, *args, **kwargs):
        self.world.apply_settings(self._settings)
    def _retrieve_data(self, sensor_queue, timeout):
        while True:
            data = sensor_queue.get(timeout=timeout)
            if data.frame == self.frame:
                return data


pygame 2.0.1 (SDL 2.0.14, Python 3.6.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


# -- BasicSynchronousClient ----------------------------------------------------

In [3]:
class BasicSynchronousClient(object):
    """
    Basic implementation of a synchronous client.
    """

    def __init__(self, args):
        self.args = args

        if not os.path.exists(args['camera_config']):
            raise AttributeError(f"Camera config file '{args['camera_config']}' is not found")

        self.copy_transforms = args['copy_transforms']
        self.image_width = args['width']
        self.image_height = args['height']

        self.type = args['type']
        self.car = None
        self.display = None
        self.location = None

        self.cameras = []

        self.vehicle_filter = args['filter']

        self.save_dir = args['savedir']

        self.camera_config = self.load_json(args['camera_config'])
        self.fov = 90
        self.focal_distance= 1000
        self.loop = 0
        self.tick = 0

        self.client = carla.Client('localhost', 2000)
        self.client.set_timeout(2.0)
        self.world = self.client.get_world()
        self.map = self.world.get_map()
        self.get_transform_init()
        self.get_transform_init2()

    def get_transform_init(self):
        self.trans_dict = {}
        self.trans_dict['camera_angle_x'] = None
        self.trans_dict['frames'] = []
    
    def get_transform_init2(self):
        self.trans_dict2 = {}
        self.trans_dict2['camera_angle_x'] = None
        self.trans_dict2['frames'] = []

    def load_list(self, file_location: str) -> list:
        with open(file_location) as file:
            spawn_locations = file.read().split('\n')
            return [int(spawn_location) for spawn_location in spawn_locations]

    def load_json(self, file_location: str) -> dict:
        """Load Json File

        Args:
            file_location (str): File location

        Returns:
            Dict: Python Dict of Read JSON File
        """
        with open(file_location) as json_file:
            return json.load(json_file)

    def image_to_base64_string(self, image):
        image_np = np.asarray(image, dtype=np.uint8)
        base64_encodded_string = base64.b64encode(image_np.flatten()).decode('utf-8')
        return base64_encodded_string

    def setup_car(self, location: carla.Location):
        # print('Setup Car')

        if self.car is not None:
            res = self.car.destroy()

        car_bp = random.choice(self.world.get_blueprint_library().filter(self.vehicle_filter))
        self.car = self.world.spawn_actor(car_bp, location)

    def camera_blueprint(self):
        """Generate Camera Bluprint

        Returns:
            carla.BlueprintLibrary: Modified Camera Blueprint
        """
        camera_bp = self.world.get_blueprint_library().find('sensor.camera.rgb')
        camera_bp.set_attribute('image_size_x', str(self.image_width))
        camera_bp.set_attribute('image_size_y', str(self.image_height))
        camera_bp.set_attribute('fov', str(self.fov))
        camera_bp.set_attribute('focal_distance', str(self.focal_distance))

        camera_segmentation_bp = self.world.get_blueprint_library().find('sensor.camera.semantic_segmentation')
        camera_segmentation_bp.set_attribute('image_size_x', str(self.image_width))
        camera_segmentation_bp.set_attribute('image_size_y', str(self.image_height))
        camera_segmentation_bp.set_attribute('fov', str(self.fov))
        return camera_bp, camera_segmentation_bp

    def get_matrix(self, x, y, z, transforms, check_x):
        if check_x:
            yaw = - (transforms.rotation.yaw - 90)
        else:
            yaw = transforms.rotation.yaw - 90
        if yaw < 0:
            yaw = 360 +yaw
        roll = 90 +transforms.rotation.pitch
        pitch = transforms.rotation.roll
        c_y = np.cos(np.radians(yaw))
        s_y = np.sin(np.radians(yaw))
        c_r = np.cos(np.radians(roll))
        s_r = np.sin(np.radians(roll))
        c_p = np.cos(np.radians(pitch))
        s_p = np.sin(np.radians(pitch))
        if check_x:
            matrix = np.identity(4)
            matrix[0, 3] = x * -1
            matrix[1, 3] = y
            matrix[2, 3] = z
            matrix[0, 0] = c_p * c_y
            matrix[0, 1] = (c_y * s_p * s_r) - (s_y * c_r)
            matrix[0, 2] = (c_y * s_p * c_r) + (s_y * s_r)
            matrix[1, 0] = s_y * c_p
            matrix[1, 1] = (s_y * s_p * s_r) + (c_y * c_r)
            matrix[1, 2] = (s_y * s_p * c_r) + (c_y * (-s_r))
            matrix[2, 0] = -s_p
            matrix[2, 1] = c_p * s_r
            matrix[2, 2] = c_p * c_r
        else:
            matrix = np.identity(4)
            matrix[0, 3] = x * -1
            matrix[1, 3] = y
            matrix[2, 3] = z
            matrix[0, 0] = c_p * c_y
            matrix[0, 1] = (c_y * s_p * s_r) - (-s_y * c_r)
            matrix[0, 2] = (c_y * s_p * c_r) + (-s_y * s_r)
            matrix[1, 0] = -s_y * c_p
            matrix[1, 1] = (s_y * s_p * s_r) + (c_y * c_r)
            matrix[1, 2] = (s_y * s_p * c_r) + (c_y * (-s_r))
            matrix[2, 0] = -s_p
            matrix[2, 1] = c_p * s_r
            matrix[2, 2] = c_p * c_r
        return matrix

    def get_transform(self, vehicle_location: carla.Location, distance: float, angle: float, pitch: float):
        """Calculate Carla Transform

        Args:
            distance (float): spherical camera distance in meters
            angle (float): camera angle in degree
            pitch (float): camera pitch in degree


        """
        a = math.radians(angle + vehicle_location.rotation.yaw)
        d = distance * math.cos(math.radians(pitch))
        h = distance * math.sin(math.radians(pitch))
        location = carla.Location(d * math.cos(a) + vehicle_location.location.x, d *
                                  math.sin(a) + vehicle_location.location.y, h + vehicle_location.location.z)

        t = carla.Transform(location, carla.Rotation(yaw=180 + angle + vehicle_location.rotation.yaw, pitch=-pitch, roll=vehicle_location.rotation.roll))
        if d*math.cos(a) >= 0:
            check_x = False
        else:
            check_x = True

        matrix = self.get_matrix(x=d*math.cos(a),
                                y=d*math.sin(a),
                                z=h,
                                transforms=t,
                                check_x = check_x)
        cam_angle_x = self.fov
        return carla.Transform(location, carla.Rotation(yaw=180 + angle + vehicle_location.rotation.yaw,
                                                        pitch=-pitch, 
                                                        roll=vehicle_location.rotation.roll)),matrix,  cam_angle_x

    def single_setup_camera(self, vehicle_location: carla.Location):
        """Initialize multi camera from self.camera_config

        Args:
            random_xs (List[float], optional): List of random x axis in meters. Defaults to None.
            random_zs (List[float], optional): List of random z axis in meters. Defaults to None.
            random_distances (List[float], optional): List of random distance in meters. Defaults to None.
        """
        camera_bp, segmentation_bp = self.camera_blueprint()

        # Camera Actor Spawning
        transform_config = self.camera_config[0]
        camera_transform, matrix, cam_angle_x = self.get_transform(
            vehicle_location, transform_config['distance'], transform_config['angle'], transform_config['pitch'])

        camera_rgb = self.world.spawn_actor(
            camera_bp,
            camera_transform)  # ,
        # attach_to=self.car)
        self.cameras.append(camera_rgb)

        segmentation_rgb = self.world.spawn_actor(
            segmentation_bp,
            camera_transform)
        self.cameras.append(segmentation_rgb)

    def get_font(self):
        fonts = [x for x in pygame.font.get_fonts()]
        default_font = 'ubuntumono'
        font = default_font if default_font in fonts else fonts[0]
        font = pygame.font.match_font(font)
        return pygame.font.Font(font, 14)

    def should_quit(self):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                return True
            elif event.type == pygame.KEYUP:
                if event.key == pygame.K_ESCAPE:
                    return True
        return False        

    def render_images(self, camera_datas, cam_idx, matrix, cam_angle_x):
        array = np.frombuffer(camera_datas[0].raw_data, dtype=np.dtype("uint8"))
        array = np.reshape(array, (camera_datas[0].height, camera_datas[0].width, 4))

        camera_datas[1].convert(carla.ColorConverter.CityScapesPalette)
        sg_array = np.frombuffer(camera_datas[1].raw_data, dtype=np.dtype("uint8"))
        sg_array = np.reshape(sg_array, (camera_datas[1].height, camera_datas[1].width, 4))
        sg_array = sg_array[:, :, :3]

        self.vehicle_mask = cv2.inRange(sg_array, (0, 0, 0), (1, 0, 0))
        _, thresh = cv2.threshold(self.vehicle_mask, 127, 255, 0)
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        sub_vehicle = np.array(array)
        sub_vehicle[self.vehicle_mask == 0] = 0
        sub_file_name = f'{self.save_dir}/{self.type}/r_{cam_idx}.png'
        cv2.imwrite(sub_file_name, sub_vehicle)

        array = array[:,:,:3]
        array = array[:, :, ::-1]
        image_surface = pygame.surfarray.make_surface(array.swapaxes(0, 1))
        self.display.blit(image_surface, (0, 0))
        
        trans_dict = {}
        trans_dict['file_path'] = f'./{self.type}/r_{cam_idx}'
        trans_dict['rotation'] = 0
        trans_dict['transform_matrix'] = list(map(list, matrix))
        self.trans_dict['frames'].append(trans_dict)
        self.trans_dict['camera_angle_x'] =cam_angle_x

    def should_quit(self):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                return True
            elif event.type == pygame.KEYUP:
                if event.key == pygame.K_ESCAPE:
                    return True
        return False

    def camera_loop(self):
        self.location = self.map.get_spawn_points()[10]
        self.setup_car(self.location)
        self.single_setup_camera(self.location)
        
        with CarlaSyncMode(self.world, *self.cameras, fps=30) as sync_mode:
            os.makedirs(f'{self.save_dir}/{self.type}', exist_ok=True)
            for cam_num, transform_config in enumerate(self.camera_config):
                if self.should_quit():
                    return True

                self.car.set_transform(self.location)
                camera_transform, matrix, cam_angle_x = self.get_transform(
                    self.location, transform_config['distance'], transform_config['angle'], transform_config['pitch'])
                # RGB Camera Transformation
                self.cameras[0].set_transform(camera_transform)
                self.cameras[1].set_transform(camera_transform)
                for i in range(10):
                    self.clock.tick()
                    data = sync_mode.tick(timeout=2.0)
                
                snapshot = data[0]
                camera_datas = data[1:]
                fps = round(1.0 / snapshot.timestamp.delta_seconds)

                self.render_images(camera_datas, cam_num, matrix, cam_angle_x)

                self.display.blit(
                    self.font.render('% 5d FPS (real)' % self.clock.get_fps(), True, (255, 255, 255)),
                    (8, 10))
                self.display.blit(
                    self.font.render('% 5d FPS (simulated)' % fps, True, (255, 255, 255)),
                    (8, 28))
                pygame.display.flip()

    def game_loop(self):
        pygame.init()

        self.display = pygame.display.set_mode(
            (self.image_width, self.image_height),
            pygame.HWSURFACE | pygame.DOUBLEBUF)
        self.font = self.get_font()
        self.clock = pygame.time.Clock()

        try:
            self.camera_loop()
            # with open(f'{self.save_dir}/transforms_{self.type}.json', 'w') as f:
            #     json.dump(self.trans_dict, f , indent=2)
            copy_transform = glob.glob(os.path.join(self.copy_transforms, '**/*.json', recursive=True))
            for i in copy_tranform:
                fn = os.path.basename(i)
                shutil.copyfile(os.path.join(self.save_dir, fn))
        finally:
            print('destroying actors.')
            for camera in self.cameras:
                camera.destroy()
            if self.car is not None:
                self.car.destroy()

        pygame.quit()
        print('done.')

In [5]:
with open('./configs/nerf/main.yaml') as k:
    args = yaml.load(k, Loader=yaml.FullLoader)
args

{'camera_config': './configs/nerf/cam_config_train.json',
 'filter': 'm1a2',
 'res': '1280x720',
 'savedir': './test_tank_img',
 'type': 'train'}

In [6]:
## test data
args['camera_config'] = './configs/nerf/cam_config_test.json'
args['type'] = 'test'
print(args)

## valid data
args['camera_config'] = './configs/nerf/cam_config_valid.json'
args['type'] = 'val'
print(args)

{'camera_config': './configs/nerf/cam_config_test.json', 'filter': 'm1a2', 'res': '1280x720', 'savedir': './test_tank_img', 'type': 'test'}
{'camera_config': './configs/nerf/cam_config_valid.json', 'filter': 'm1a2', 'res': '1280x720', 'savedir': './test_tank_img', 'type': 'val'}


In [ ]:
args['width'], args['height'] = [int(x) for x in args['res'].split('x')]
try:
    client = BasicSynchronousClient(args)
    client.game_loop()
except KeyboardInterrupt:
    print('Interuptted')
finally:
    print('EXIT')